In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import os ; os.environ['OMP_NUM_THREADS'] = '4'

# Any results you write to the current directory are saved as output.

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import numpy as np
import operator
import random
import datetime


from lightgbm import LGBMRegressor
import sklearn.discriminant_analysis
import sklearn.linear_model as skl_lm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from datetime import timedelta
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import hamming_loss, accuracy_score 
from pandas import DataFrame
from datetime import datetime
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

%matplotlib inline
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
PATH ='/kaggle/input/covid19-global-forecasting-week-5'
datatrain = pd.read_csv(f'{PATH}/train.csv')
datatest = pd.read_csv(f'{PATH}/test.csv')

In [ ]:
datatrain.head()

In [ ]:
datatrain.describe()

In [ ]:
datatrain.info()

In [ ]:
date = pd.to_datetime(datatrain["Date"])
datet = pd.to_datetime(datatest["Date"])
print (date)
datatrain["Date"] = date
datatest["Date"] = datet

In [ ]:
datatrain["Date"]

In [ ]:
ldate = int(len(date))
ldatet = int(len(datet))
print("Length of training- date is", ldate)
print("Length of test- date is", ldatet)

In [ ]:
m = []
d = []
for i in range(0,ldate):
    dx = (date[i].strftime("%d"))
    mx = (date[i].strftime("%m"))
    m.append(int(mx))
    d.append(int(dx))

mt = []
dt = []
for i in range(0,ldatet):
    dtx = (datet[i].strftime("%d"))
    mtx = (datet[i].strftime("%m"))
    mt.append(int(mtx))
    dt.append(int(dtx))


In [ ]:
train = datatrain
test = datatest


## Added Input of Month + Day from Date

In [ ]:
train.insert(6,"Month",m,False)
train.insert(7,"Day",d,False)
test.insert(4,"Month",mt,False)
test.insert(5,"Day",dt,False)

In [ ]:
train.head()

In [ ]:
train.tail(20)

In [ ]:
test.head()

In [ ]:
print("Datatrain -------------------------------------")
traindays = datatrain['Date'].nunique()
print("Number of Country_Region: ", datatrain['Country_Region'].nunique())
print("Number of Province_State: ", datatrain['Province_State'].nunique())
print("Number of Days: ", traindays)

notrain = datatrain['Id'].nunique()
print("Number of datapoints in train:", notrain)
lotrain = int(notrain/traindays)
print("L Trains:", lotrain)


In [ ]:
print("Datatest -------------------------------------")
testdays = datatest['Date'].nunique()
print("Number of Country_Region: ", datatest['Country_Region'].nunique())
print("Number of Province_State: ", datatest['Province_State'].nunique())
print("Number of Days: ", testdays)
notest = datatest['ForecastId'].nunique()
print("Number of datapoints in test:", notest)
lotest = int(notest/testdays)
print("L Test:", lotest)


In [ ]:
train['Target2'] = train.Target.factorize()[0]
test['Target2'] = test.Target.factorize()[0]

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
traincount = int(len(train["Date"]))
testcount = int(len(test["Date"]))

In [ ]:
train.Province_State = train.Province_State.fillna(0)
train.County = train.County.fillna(0)
empty = 0
for i in range(0,traincount):
    if(train.Province_State[i] == empty):
        train.Province_State[i] = train.Country_Region[i];
   

In [ ]:
for i in range(0,traincount):
    if(train.County[i] == empty):
        train.County[i] = train.Province_State[i];

In [ ]:
train.head()

In [ ]:
train["Pop"] = np.log1p(train["Population"])
test["Pop"] = np.log1p(test["Population"])

In [ ]:
train.head()

In [ ]:
test.Province_State = test.Province_State.fillna(0)
test.County = test.County.fillna(0)
empty = 0
for i in range(0,testcount):
    if(test.Province_State[i] == empty):
        test.Province_State[i] = test.Country_Region[i];


In [ ]:
empty=0
for i in range(0,testcount):    
    if(test.County[i] == 0):
        test.County[i] = test.Province_State[i];

In [ ]:
test.head()

In [ ]:
label = preprocessing.LabelEncoder()
train.Country_Region = label.fit_transform(train.Country_Region)
train.Province_State = label.fit_transform(train.Province_State)
train.County = label.fit_transform(train.County)
train.head()

In [ ]:
test.Country_Region = label.fit_transform(test.Country_Region)
test.Province_State = label.fit_transform(test.Province_State)
test.County = label.fit_transform(test.County)
test.head()

In [ ]:
c_train = train.loc[train['Target'] == 'ConfirmedCases']
f_train = train.loc[train['Target'] == 'Fatalities']

c_test = test.loc[test['Target'] == 'ConfirmedCases']
f_test = test.loc[test['Target'] == 'Fatalities']

td = datet[0]
tdate = c_train.loc[c_train['Date'] == td]
c_train.head(90)

In [ ]:
datet

In [ ]:
f_train.tail(10)

In [ ]:
zt = datet[0]
daycount = []
lt = int(lotrain/2) #Since the dataset is divided into confirmed and fatalities, lotrain is divided by 2 since there are 2 targets.
for i in range(0,lt):
    for j in range(1,traindays+1):
        daycount.append(j)


In [ ]:
len(daycount)

In [ ]:
tx = traindays * 2
zo = []

for i in range(0,tx):
    if(datet[0] == date[i]):
        zx = i
        zo.append(zx)


In [ ]:
print(zo)

In [ ]:
zx = zo[0]/2
print(zx)

In [ ]:
daytest = []
lt2 = int(lotest/2)
for i in range(0,lt2):
    for j in range(1,testdays+1):
        jr = zx + j
        daytest.append(jr)


In [ ]:
len(daytest)

In [ ]:
daytest

In [ ]:
c_train.head()

In [ ]:
f_train.head()

In [ ]:
print("Length of Date in train:",len(c_train["Date"]))
print("Length of Daycount:",len(daycount))

## Added DayCount as Input from Dates.

In [ ]:
c_train.insert(8,"DayCount",daycount,False)
c_test.insert(6,"DayCount",daytest,False)

In [ ]:
c_test.head()

In [ ]:
f_train.insert(8,"DayCount",daycount,False)

In [ ]:
f_test.insert(6,"DayCount",daytest,False)
f_train.head()

In [ ]:
f_test.head()

In [ ]:
cX = np.c_[c_train["County"], c_train["Province_State"], c_train["Country_Region"], c_train["DayCount"],c_train["Pop"], c_train["Weight"]]
fX = np.c_[f_train["County"], f_train["Province_State"], f_train["Country_Region"], f_train["DayCount"],f_train["Pop"], f_train["Weight"]]

cXt = np.c_[c_test["County"], c_test["Province_State"], c_test["Country_Region"], c_test["DayCount"], c_test["Pop"],c_test["Weight"]]
fXt = np.c_[f_test["County"], f_test["Province_State"], f_test["Country_Region"], f_test["DayCount"], f_test["Pop"], f_test["Weight"]]


In [ ]:
cy = c_train["TargetValue"]
fy = f_train["TargetValue"]

In [ ]:
regr = XGBRegressor(n_estimators = 2600 , alpha = 0, gamma = 0, learning_rate = 0.04,  random_state = 42 , max_depth = 25, nthread = 4) 
regr1 = XGBRegressor(n_estimators = 2600 , alpha = 0, gamma = 0, learning_rate = 0.04,  random_state = 42 , max_depth = 25, nthread = 4 ) 

In [ ]:
regr.fit(cX,cy.ravel())
yscore = regr.predict(cX)
yscore = np.round(yscore)
ascore = mean_squared_error(yscore,cy)
print("Training (ConfirmedCases) - Mean Squared Error is: ",ascore)

In [ ]:
regr1.fit(fX,fy.ravel())
yscore = regr1.predict(fX)
yscore = np.round(yscore)
ascore = mean_squared_error(yscore,fy)
print("Training (Fatalities) - Mean Squared Error is: ",ascore)

In [ ]:
#regr = XGBRegressor(n_estimators = 2300 , alpha = 0, gamma = 0, learning_rate = 0.04,  random_state = 42 , max_depth = 23) #Training (ConfirmedCases) - Mean Squared Error is:  0.0005750290495952613
#regr1 = XGBRegressor(n_estimators = 2300 , alpha = 0, gamma = 0, learning_rate = 0.04,  random_state = 42 , max_depth = 23)

#regr = XGBRegressor(n_estimators = 2300 , alpha = 0, gamma = 0, learning_rate = 0.03,  random_state = 42 , max_depth = 23) # Best
#regr1 = XGBRegressor(n_estimators = 2300 , alpha = 0, gamma = 0, learning_rate = 0.03,  random_state = 42 , max_depth = 23) 

In [ ]:
#regr = XGBRegressor(n_estimators = 2250 ,min_child_weight = 1, gamma = 1.15, learning_rate = 0.035,  random_state = 42 , max_depth = 23) # 0.0055
#regr1 = XGBRegressor(n_estimators = 2250 ,min_child_weight = 1, gamma = 1.15, learning_rate = 0.035,  random_state = 42 , max_depth = 23) 

In [ ]:
#regr = XGBRegressor(n_estimators = 2300 , alpha = 0, gamma = 0, learning_rate = 0.03,  random_state = 42 , max_depth = 23) # Best
#regr1 = XGBRegressor(n_estimators = 2300 , alpha = 0, gamma = 0, learning_rate = 0.03,  random_state = 42 , max_depth = 23)

In [ ]:
ypred = regr.predict(cXt)
ypred = np.round(ypred)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred.head(20) 

In [ ]:
ypred.tail(20)

In [ ]:
ypred2 = regr1.predict(fXt)
ypred2 = np.round(ypred2)

In [ ]:
ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2.head(20) # 6 6 7 7 11

In [ ]:
ypred2.tail(20)

In [ ]:
len(ypred)

In [ ]:
ypc = pd.DataFrame()
forecast = c_test["ForecastId"]
ypc.insert(0,"ForecastId",forecast,False)
ypc.reset_index(drop=True, inplace=True)
ypc.insert(1,"TargetValue",ypred,False)
ypc.head()

In [ ]:
len(ypc)

In [ ]:
ypc.tail()

In [ ]:
forecast2 = f_test["ForecastId"]
ypf = pd.DataFrame()
ypf.insert(0,"ForecastId",forecast2,False)
ypf.reset_index(drop=True, inplace=True)
ypf.insert(1,"TargetValue",ypred2,False)
ypf.head()

In [ ]:
len(ypf)

In [ ]:
ypf.tail()

In [ ]:
a=ypc.groupby(['ForecastId'])['TargetValue'].quantile(q=0.05).reset_index()
b=ypc.groupby(['ForecastId'])['TargetValue'].quantile(q=0.5).reset_index()
c=ypc.groupby(['ForecastId'])['TargetValue'].quantile(q=0.95).reset_index()

a.columns=['Id','q0.05']
b.columns=['Id','q0.5']
c.columns=['Id','q0.95']
a=pd.concat([a,b['q0.5'],c['q0.95']],1)
a

In [ ]:
a2=ypf.groupby(['ForecastId'])['TargetValue'].quantile(q=0.05).reset_index()
b2=ypf.groupby(['ForecastId'])['TargetValue'].quantile(q=0.5).reset_index()
c2=ypf.groupby(['ForecastId'])['TargetValue'].quantile(q=0.95).reset_index()

a2.columns=['Id','q0.05']
b2.columns=['Id','q0.5']
c2.columns=['Id','q0.95']
a2=pd.concat([a2,b2['q0.5'],c2['q0.95']],1)
a2

In [ ]:
sub=pd.melt(a, id_vars=['Id'], value_vars=['q0.05','q0.5','q0.95'])
sub['variable']=sub['variable'].str.replace("q","", regex=False)
sub['ForecastId_Quantile']=sub['Id'].astype(str)+'_'+sub['variable']
sub['TargetValue']=sub['value']
sub=sub[['ForecastId_Quantile','TargetValue']]
sub.reset_index(drop=True,inplace=True)
sub1 = sub.copy()
sub1

In [ ]:
sub=pd.melt(a2, id_vars=['Id'], value_vars=['q0.05','q0.5','q0.95'])
sub['variable']=sub['variable'].str.replace("q","", regex=False)
sub['ForecastId_Quantile']=sub['Id'].astype(str)+'_'+sub['variable']
sub['TargetValue']=sub['value']
sub=sub[['ForecastId_Quantile','TargetValue']]
sub.reset_index(drop=True,inplace=True)
sub2 = sub.copy()
sub2

In [ ]:
sub = sub1.append(sub2)
sub

In [ ]:
sub.reset_index(drop=True,inplace=True)
sub.to_csv("submission.csv",index=False)
sub.head()

In [ ]:
sub.tail()